In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
import os
root_path = 'gdrive/My Drive/EACL/'
os.chdir(root_path)

Mounted at /content/gdrive


In [2]:
!pip install sentencepiece
!pip install transformers
!pip install demoji

     |████████████████████████████████| 1.2MB 21.5MB/s 
     |████████████████████████████████| 1.8MB 16.3MB/s 
     |████████████████████████████████| 2.9MB 39.1MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c88ea33a6ef91463cfea6b1f7ec60b0ad5b305c32fe6d785e1de49dda995099d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!pip install git+git://github.com/irshadbhat/indic-trans.git

  Cloning git://github.com/irshadbhat/indic-trans.git to /tmp/pip-req-build-9eyfqoqd
  Running command git clone -q git://github.com/irshadbhat/indic-trans.git /tmp/pip-req-build-9eyfqoqd
  Using cached https://files.pythonhosted.org/packages/fb/48/69046506f6ac61c1eaa9a0d42d22d54673b69e176d30ca98e3f61513e980/pbr-5.5.1-py2.py3-none-any.whl
  Created wheel for indictrans: filename=indictrans-1.2.3-cp36-cp36m-linux_x86_64.whl size=337563794 sha256=cdb67cf0494028ae8d969452329b8a4c1e7d6ca92aa1c398f8af23729dd41c12
  Stored in directory: /tmp/pip-ephem-wheel-cache-uovib90c/wheels/01/a9/02/ca41463296c4ea023505525a9e355c68f3a71225c81495e6d7
Successfully built indictrans


In [4]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import copy
from transformers import BertModel, RobertaModel, BertTokenizer, RobertaTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score 
from tqdm import tqdm
import demoji 
from indictrans import Transliterator
demoji.download_codes() 
plt.rcParams['figure.figsize'] = [15, 8]
plt.rcParams.update({'font.size': 8})
RANDOM_SEED = 42
path = Path('./')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

... OK (Got response in 0.13 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [5]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.7.0+cu101')

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
random_seed(RANDOM_SEED, True)

In [7]:
def handle_all_caps(t: str) -> str:
    tokens = t.split()
    tokens = replace_all_caps(tokens)
    return ' '.join(tokens)

def handle_upper_case_first_letter(t: str) -> str:
    tokens = t.split()
    tokens = deal_caps(tokens)
    return ' '.join(tokens)

def lower_case_everything(t: str) -> str:
    return t.lower().replace('@user', '').replace('#tag ', '').replace('rt ', '').strip()

In [8]:
class CodeMixedTamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str(path/"./Tamil/taen_spm.model"))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

sp = spm.SentencePieceProcessor()
sp.Load(str(path/"./Tamil/taen_spm.model"))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [9]:
itos[:20]

['xxunk',
 'xxbos',
 'xxeos',
 'xxpad',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 '.',
 ',',
 '▁',
 's',
 'a',
 '="',
 'in',
 'doc',
 't',
 'il',
 'i']

In [10]:
class Dataset():
    def __init__(self, train_data, val_data, test_data):
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        # self.batch_size = batch_size

        self.label_dict = {}
        
        self.count_dic = {}

        self.trn = Transliterator(source='tam', target='eng', build_lookup=True)

        self.train_df = pd.DataFrame(self.process_data(self.train_data))
        self.val_df = pd.DataFrame(self.process_data(self.val_data))
        self.test_df = pd.DataFrame(self.process_data(self.test_data, test = True))

    def is_english(self, s: str) -> bool:
        if len(re.findall(u'[\u0900-\u097F]', s)) <= 1000:
            return True
        return False

    def process_data(self, data, test = False):
        sentences, labels = [], []
        print(len(data))
        for line in data:
            sentence = line.strip().split('\t')
            if not test:
                label = sentence.pop()
                if label not in self.label_dict:
                    self.label_dict[label] = len(self.label_dict)
                labels.append(self.label_dict[label])
                self.count_dic[label] = self.count_dic.get(label, 0) + 1
            sentence = ' '.join(sentence)
            emoji_dict = demoji.findall(sentence)

            # if not self.is_english(sentence):
            #     print(sentence)
            #     sentence = self.trn.transform(sentence)
            #     print(sentence)

            sentence = self.trn.transform(sentence)

            if len(emoji_dict): 
                for emoji, text in emoji_dict.items():
                    sentence = sentence.replace(emoji, ' '+text+' ')
                    sentence = ' '.join(sentence.split())
            
            sentences.append(sentence)
            
            # labels.append(label)
            
        if test: return {'input': sentences}
        return {'input': sentences, 'label': (labels)}

In [11]:
with open('Dataset/tamil_offensive_full_train.csv', 'r') as f:
    train_data = f.readlines()
with open('Dataset/tamil_offensive_full_dev.csv', 'r') as f:
    val_data = f.readlines()
with open('Dataset/tamil_offensive_full_test.csv', 'r') as f:
    test_data = f.readlines()
data = Dataset(train_data, val_data, test_data)

35139
4388
4392


In [12]:
data.label_dict

{'Not_offensive': 0,
 'Offensive_Targeted_Insult_Group': 3,
 'Offensive_Targeted_Insult_Individual': 5,
 'Offensive_Targeted_Insult_Other': 2,
 'Offensive_Untargetede': 4,
 'not-Tamil': 1}

In [13]:
data.test_df.head()

,input
0,14.12.2018 epo trailer pathutu irken ... Semay...
1,Paka thana poro movie la Enna irukunu
2,“U kena tunggu lebih lama lagi untuk tahu saya...
3,Suriya anna vera level anna mass
4,suma kaththaatha da sound over a pooda kudaath...


In [14]:
from collections import Counter
Counter(data.train_df['label'])

Counter({0: 25425, 1: 1454, 2: 454, 3: 2557, 4: 2906, 5: 2343})

In [15]:
label_cols = ['label']
text_cols = ['input']

In [16]:
taen_vocab = Vocab(itos)
tokenizer = Tokenizer(lang='taen', tok_func=CodeMixedTamilTokenizer)
tokenizer.pre_rules.append(lower_case_everything)
tokenizer.pre_rules.append(handle_all_caps)
tokenizer.pre_rules.append(handle_upper_case_first_letter)

In [17]:
tokenizer.special_cases, tokenizer.pre_rules, tokenizer.post_rules

(['xxunk',
  'xxpad',
  'xxbos',
  'xxeos',
  'xxfld',
  'xxmaj',
  'xxup',
  'xxrep',
  'xxwrep'],
 [<function fastai.text.transform.fix_html>,
  <function fastai.text.transform.replace_rep>,
  <function fastai.text.transform.replace_wrep>,
  <function fastai.text.transform.spec_add_spaces>,
  <function fastai.text.transform.rm_useless_spaces>,
  <function __main__.lower_case_everything>,
  <function __main__.handle_all_caps>,
  <function __main__.handle_upper_case_first_letter>],
 [<function fastai.text.transform.replace_all_caps>,
  <function fastai.text.transform.deal_caps>])

In [18]:
tokens = tokenizer.process_all(['Tell me about TOUR self, mujhe jaanna hai'])
''.join(tokens[0])

'▁tell▁me▁about▁tour▁self,▁mujhe▁jaanna▁hai'

In [19]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=data.train_df, valid_df=data.val_df, test_df=data.test_df, tokenizer=tokenizer, vocab=taen_vocab, label_cols=label_cols, text_cols=text_cols)

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [20]:
data_lm.show_batch()

/usr/local/lib/python3.6/dist-packages/fastai/text/data.py:339: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx_min = (t != self.pad_idx).nonzero().min()


idx,text
0,▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . . ▁x x bo s ▁count er ▁devar . char pa uk ▁vellri ▁peni ▁va hat us ▁li on ▁x x bo s ▁ip po ▁in tha ▁tra il er ▁ ah ▁park ura vana ▁oru ▁ like ▁po
1,". . . ▁x x bo s ▁varum ▁thi rai ▁ yang um ▁e tu thi kkum ▁pa ya varu kirar ▁s 3 ▁x x bo s ▁la u gh ▁ri ot ▁f rom ▁0 . 24 ▁dai ▁kel ava ▁x x bo s ▁un galuku ▁ver a ▁vel aya ▁il aya . . sam e ▁s tor y ▁ , ▁sam e ▁sc ri p t . ▁ad vi"
2,▁por om ? ? ▁tha lai var : s we et ▁sa ap ida ▁por om . . . ▁ver a ▁le vel . . . ▁x x bo s ▁at li ▁math i ri ▁mutt al u kal ▁kan du ▁padi kkat tum ▁e ppa di ▁padam ▁pud ikk anam ▁en tra th ▁x x bo s ▁vay as ukku ▁tha gu ndha ▁kel vi ▁ya ▁ke kka ▁sol
3,xxrep ▁6 ▁ y ▁ye ah ▁ but ▁ay tha ▁e z hu th u ▁mari ▁irukk ke y ▁x x bo s ▁mani ▁sir ▁com eb ack ▁tri ler ▁mas s ▁ver a ▁le val ▁x x bo s ▁ena ▁da ▁in tha ▁2 ▁min s ▁e h ▁f ull ▁ ah ▁pa ak a ▁mudi la ▁x x bo s ▁edu ku ▁dis like ▁pan r avan ▁el
4,ap en ▁x x bo s ▁re mak e ▁of ▁the ▁sho f il m ▁ke kka ▁be kka ▁ke kka ▁be kka ▁x x bo s ▁de a sar ▁on ly ▁dal ▁mo vi e ▁than ▁mas s ▁x x bo s ▁dis like s ▁pan rav ag a ▁ lam ▁na ada ga ▁ka ad hal ▁ku mbal s ▁ xxrep ▁4 ▁ . ▁x x bo s


In [ ]:
# data_lm.vocab.itos = data_lm.vocab.itos
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, pretrained=False, model_dir = './Tamil/models/')

In [ ]:
learn.load('best_model',with_opt=True, device=device)


LanguageLearner(data=TextLMDataBunch;

Train: LabelList (35139 items)
x: LMTextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁count er ▁devar . char pa uk ▁vellri ▁peni ▁va hat us ▁li on
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (4388 items)
x: LMTextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁thenkashi ▁mawatam ▁nad ar ▁samudaya m ▁charpauk ▁va hat us,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁sicipu ▁ xxrep ▁5 ▁ . ▁melum ▁itu ▁bonthe ▁patippues ▁mik ▁avasium ▁ xxrep ▁4 ▁ .,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . ▁se m ma ▁tra il er . ▁c ros s ed ▁f ing ers
y: LMLabelLi

In [ ]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.save('fit_head', with_opt=True)

In [ ]:
learn.load('fit_head', with_opt=True)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.764874,3.689709,0.376600,00:57
1,3.481884,3.446873,0.407493,00:56
2,3.274419,3.344945,0.420268,00:57
3,3.191977,3.308245,0.425670,00:57
4,3.155731,3.303349,0.426064,00:57


In [ ]:
learn.save('fine_tuned', with_opt=True)

'en payar ▁count arai ▁samudaya m ▁charpauk ▁kull ull ▁devar attam ▁ippattin'

In [ ]:
learn.load('fine_tuned', with_opt=True);
learn.predict('en payar',n_words=10)

'en payar ▁ke dai dra v aiye ▁cholli va ndar o ▁ange'

In [ ]:
learn.save_encoder('fine_tuned_enc')

In [ ]:
def change_1h_targ(targ):
    n = len(data.label_dict)
    ohe = [0]*n
    ohe[targ] = 1
    return ohe

In [ ]:
data.train_df['ohe'] = data.train_df['label'].apply(change_1h_targ)
data.val_df['ohe'] = data.val_df['label'].apply(change_1h_targ)
label_cols = ['label']
text_cols = ['input']

In [ ]:
data.train_df

,input,label,ohe
0,movie vara level la Erika poguthu,0,"[1, 0, 0, 0, 0, 0]"
1,I love Ajith Kumar Vivegam movie inki mjy bht ...,1,"[0, 1, 0, 0, 0, 0]"
2,Padam nalla comedy padama irukum polaye..,0,"[1, 0, 0, 0, 0, 0]"
3,karthick subburaj anne .... intha padam vetri ...,0,"[1, 0, 0, 0, 0, 0]"
4,counter devar.charpauk vellri peni vahatus lion,0,"[1, 0, 0, 0, 0, 0]"
...,...,...,...
35134,Trending number #2 idhukku nammalam karanamnu ...,0,"[1, 0, 0, 0, 0, 0]"
35135,"Movie script super, athuvum HIP HOP Tamizha mu...",0,"[1, 0, 0, 0, 0, 0]"
35136,Just 3k likes for 300k likes,0,"[1, 0, 0, 0, 0, 0]"
35137,Aaloo le lo. Kanda le lo.,1,"[0, 1, 0, 0, 0, 0]"


In [21]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=data.train_df, valid_df=data.val_df, test_df = data.test_df, tokenizer=tokenizer, vocab=taen_vocab, bs=16, label_cols = label_cols, text_cols=text_cols)
data_clas.show_batch()

/usr/local/lib/python3.6/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


text,target
▁x x bo s ▁tira u pati ▁ xxrep ▁10 ▁= ▁mann es ▁penn es ▁ka kk ▁vant ▁ nayaki ▁ xxrep ▁4 ▁ . ▁naatak ▁kadal ai ▁tol ur ikum ▁navina ▁tira u pati ▁ xxrep ▁5 ▁ . ▁pin chil ▁badutt a ▁we m pi ▁than ▁vat um ▁nan jai ▁suva ittal ▁maranam ▁than ▁min ch m ▁ xxrep ▁5 ▁ . ▁per si var uku ▁than ▁theri,3
"▁x x bo s ▁10 ▁( aver ag e ) ▁pon n ung ▁viddu ku ▁na ck y ▁ton k ▁botu ttu ▁ivan uk ▁pon tu , ▁aval u galum ▁sir ie ▁sing ari ch ittu ▁10 ▁ambal ing ▁munn adi ▁foi ▁bo se ▁tar itu . ▁ivan uk ▁evvalou ▁var ad t chan ai ▁taruv anu ▁kat k antu ▁aval uk ▁evvalou ▁syettu ▁va chir ukkan ▁ / ▁chamba",3
▁x x bo s ▁it ▁ot uki du ▁vill uppu nar u ▁padam ▁kadu etty ar ▁dam pia ▁ka kar om ▁varu ngal adu l ▁edu ng ▁arasu ▁turai l ▁vanni er ▁devar ▁ka un der ▁vell aur ▁yadav ar ▁chett ar nu ▁nu ▁ell ▁samudaya mum ▁bu lli nga ▁pati kk il tu l ▁irundhu ▁ella du kum ▁poor vi k ▁chott ▁vi c rom ▁nam ▁vari ppan m,0
▁x x bo s ▁0 : 59 ▁un k n ow n ▁ num ber ▁ ; ▁en da ▁manda ya ▁mara ch een ga ▁konda ya ▁mara ▁che e ngal a ; ▁ num ber ▁is ▁vis i ble ▁ro l ling ▁on ▁the ▁flo or ▁la u gh ing ▁ro l ling ▁on ▁the ▁flo or ▁la u gh ing ▁ro l ling ▁on ▁the ▁flo or ▁la,0
"▁x x bo s ▁than ks ▁pa ▁in tham ari ▁padam ▁edu th ath ukku ▁nam m ave ttu ▁pilla i gal ukku ▁eth u ▁oru ▁se ma ▁padam ▁ , ath uv um ▁ant ha ▁inter vi e w ▁la ▁son ning a ▁paru nga ▁the ater ▁la ▁po i ▁par ung anu ▁solla ▁do w n lo ad ▁pann iya ch um ▁par ung anu ▁se ma ▁pa",0


In [22]:
data_clas.sanity_check()

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [23]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, model_dir = './Tamil/models')
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (35139 items)
x: TextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁count er ▁devar . char pa uk ▁vellri ▁peni ▁va hat us ▁li on
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (4388 items)
x: TextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁thenkashi ▁mawatam ▁nad ar ▁samudaya m ▁charpauk ▁va hat us,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁sicipu ▁ xxrep ▁5 ▁ . ▁melum ▁itu ▁bonthe ▁patippues ▁mik ▁avasium ▁ xxrep ▁4 ▁ .,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . ▁se m ma ▁tra il er . ▁c ros s ed ▁f ing ers
y: CategoryLi

In [ ]:
learn.freeze()
learn.loss_func.func

CrossEntropyLoss()

In [ ]:
f1 = FBeta(beta = 1,average = 'weighted')

In [ ]:
learn.metrics = [f1]

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,f_beta,time
0,0.809048,0.735917,0.676067,00:47


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.save('first-full')

In [ ]:
learn.load('first-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (35139 items)
x: TextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁count er ▁devar . char pa uk ▁vellri ▁peni ▁va hat us ▁li on
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (4388 items)
x: TextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁thenkashi ▁mawatam ▁nad ar ▁samudaya m ▁charpauk ▁va hat us,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁sicipu ▁ xxrep ▁5 ▁ . ▁melum ▁itu ▁bonthe ▁patippues ▁mik ▁avasium ▁ xxrep ▁4 ▁ .,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . ▁se m ma ▁tra il er . ▁c ros s ed ▁f ing ers
y: CategoryLi

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,f_beta,time
0,0.721922,0.672188,0.718449,00:55


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn.save('second-full')

In [ ]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (35139 items)
x: TextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁count er ▁devar . char pa uk ▁vellri ▁peni ▁va hat us ▁li on
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (4388 items)
x: TextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁thenkashi ▁mawatam ▁nad ar ▁samudaya m ▁charpauk ▁va hat us,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁sicipu ▁ xxrep ▁5 ▁ . ▁melum ▁itu ▁bonthe ▁patippues ▁mik ▁avasium ▁ xxrep ▁4 ▁ .,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . ▁se m ma ▁tra il er . ▁c ros s ed ▁f ing ers
y: CategoryLi

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='f_beta', name='final')])

epoch,train_loss,valid_loss,f_beta,time
0,0.617821,0.656613,0.739116,02:02
1,0.659365,0.642266,0.754676,01:53
2,0.640485,0.642448,0.749290,01:49
3,0.585432,0.643950,0.756755,01:58
4,0.510516,0.643676,0.758727,01:50


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 0 with f_beta value: 0.7391160726547241.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 1 with f_beta value: 0.7546758651733398.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ar

Better model found at epoch 3 with f_beta value: 0.7567550539970398.


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Better model found at epoch 4 with f_beta value: 0.7587268948554993.


In [24]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (35139 items)
x: TextList
▁x x bo s ▁mo vi e ▁var a ▁le vel ▁la ▁eri ka ▁po gu th u,▁x x bo s ▁i ▁lo ve ▁aj ith ▁kumar ▁viv e gam ▁mo vi e ▁in ki ▁m j y ▁ bh t ▁achi ▁l gi,▁x x bo s ▁padam ▁nall a ▁com ed y ▁padam a ▁iru kum ▁pol aye . .,▁x x bo s ▁kar thi ck ▁su bbu raj ▁ann e ▁ xxrep ▁4 ▁ . ▁in tha ▁padam ▁ve tri ▁ada ya ▁un a gal ukku ▁en n uda ya ▁val th ukk al . . .,▁x x bo s ▁count er ▁devar . char pa uk ▁vellri ▁peni ▁va hat us ▁li on
y: CategoryList
0,1,0,0,0
Path: .;

Valid: LabelList (4388 items)
x: TextList
▁x x bo s ▁han ds ome ▁hu n k ▁ke ri ▁va a ▁tha lai va a,▁x x bo s ▁thenkashi ▁mawatam ▁nad ar ▁samudaya m ▁charpauk ▁va hat us,▁x x bo s ▁je ▁ vo us ▁ai me ▁br avo ▁po ur ▁cli p ▁de ▁mer de ▁qu e ▁j ▁ xxunk co u te ▁au ▁to il ette .,▁x x bo s ▁sicipu ▁ xxrep ▁5 ▁ . ▁melum ▁itu ▁bonthe ▁patippues ▁mik ▁avasium ▁ xxrep ▁4 ▁ .,▁x x bo s ▁ver a ▁le vel ▁b g m ▁ . . ▁se m ma ▁tra il er . ▁c ros s ed ▁f ing ers
y: CategoryLi

In [25]:
def get_predicted(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat

df_test = data.val_df.copy()
from sklearn.metrics import f1_score,accuracy_score
df_dict = {'query': list(df_test['input']), 'actual_label': list(df_test['label']), 'predicted_label': [0]*df_test.shape[0]}  
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)
df_result['predicted_label'] = list(get_predicted(np.array(preds[0])))
df_result.head()

,query,actual_label,predicted_label
0,Handsome hunk keri vaa thalaivaa,0,0
1,thenkashi mawatam nadar samudayam charpauk vah...,0,0
2,je vous aime bravo pour clip de merde que j éc...,1,1
3,sicipu..... melum itu bonthe patippues mik ava...,0,0
4,Vera level BGM .. semma trailer. crossed fingers,0,0


In [26]:
f1_score(df_result['actual_label'], df_result['predicted_label'], average = 'weighted')

0.7587268628954762

In [ ]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7830446672743847

In [27]:
logits = {}
preds = learn.get_preds(ds_type=DatasetType.Valid, ordered=True)
logits['val'] = preds[0]
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
logits['test'] = preds[0]

In [28]:
with open('tam_ulmfit_preds.pkl', 'wb') as f:
    pickle.dump(logits, f)

In [29]:
with open('tam_ulmfit_preds.pkl', 'rb') as f:
    dpred = pickle.load(f)

In [30]:
dpred

{'test': tensor([[9.7739e-01, 1.2637e-05, 4.7476e-04, 1.6637e-03, 1.8689e-02, 1.7674e-03],
         [7.1744e-01, 1.3114e-04, 1.9899e-02, 4.1196e-02, 2.0459e-01, 1.6744e-02],
         [1.8518e-01, 8.1388e-01, 2.0480e-04, 1.8708e-04, 3.3984e-04, 2.0586e-04],
         ...,
         [3.7247e-01, 1.4980e-03, 3.5686e-02, 2.7768e-01, 2.8655e-01, 2.6119e-02],
         [7.2332e-01, 8.0857e-03, 1.4075e-02, 1.0035e-01, 2.9068e-02, 1.2511e-01],
         [7.6852e-02, 1.3278e-02, 5.5149e-02, 3.3028e-01, 3.0184e-01, 2.2260e-01]]),
 'val': tensor([[9.4630e-01, 2.5260e-02, 2.2303e-03, 4.5569e-03, 1.7176e-02, 4.4743e-03],
         [9.9871e-01, 1.1815e-04, 7.0968e-05, 1.9295e-04, 5.7516e-04, 3.2838e-04],
         [1.1492e-01, 8.4584e-01, 3.3452e-03, 5.3570e-03, 1.2424e-02, 1.8113e-02],
         ...,
         [1.2351e-01, 3.7066e-03, 3.0324e-02, 3.9621e-01, 3.8728e-01, 5.8961e-02],
         [9.8717e-01, 4.8754e-03, 6.6320e-04, 2.3366e-03, 7.6715e-04, 4.1830e-03],
         [9.8976e-01, 2.5471e-04, 1.0145e-